In [1]:
import os
os.chdir('/media/term/MISC/Malware/Malimg/')
print os.getcwd()


/media/term/MISC/Malware/Malimg


In [2]:
import os,glob,numpy
# the parent folder
os.chdir('malimg_dataset_imgs') 
# python list of strings with family names
list_fams = os.listdir(os.getcwd()) 
# list storing number of samples per family
no_imgs = [] 

for i in range(len(list_fams)):
 os.chdir(list_fams[i])
 len1 = len(glob.glob('*.png')) # images are stored as 'png'
 no_imgs.append(len1)
 os.chdir('..')

total = sum(no_imgs) # total number of all samples
y = numpy.zeros(total) # creating a label vector

temp1 = numpy.zeros(len(no_imgs)+1)
temp1[1:len(temp1)]=no_imgs
temp2 = int(temp1[0]); # now temp2 is [0 no_imgs]

for jj in range(len(no_imgs)): 
    temp3 = temp2 +int(temp1[jj+1])
    for ii in range(temp2,temp3): 
       y[ii] = jj
    temp2 = temp2+ int(temp1[jj+1])


In [3]:
import numpy as np
numpy.save("list_fams.npy",list_fams)

In [4]:
from PIL import Image
import leargist 

X = numpy.zeros((sum(no_imgs),320)) # Feature Matrix
cnt = 0
for i in range(len(list_fams)):
   os.chdir(list_fams[i])
   print os.getcwd()
   Image.Image.tostring = Image.Image.tobytes
   img_list = glob.glob('*.png')
   for j in range(len(img_list)):
    im = Image.open(img_list[j])
    # Reducing Image size for faster computation
    im1 = im.resize((64,64),Image.ANTIALIAS)
    des = leargist.color_gist(im1)
    X[cnt] = des[0:320]
    cnt = cnt + 1 
   os.chdir('..')

/media/term/MISC/Malware/Malimg/malimg_dataset_imgs/Adialer.C
/media/term/MISC/Malware/Malimg/malimg_dataset_imgs/Agent.FYI
/media/term/MISC/Malware/Malimg/malimg_dataset_imgs/Allaple.A
/media/term/MISC/Malware/Malimg/malimg_dataset_imgs/Allaple.L
/media/term/MISC/Malware/Malimg/malimg_dataset_imgs/Alueron.gen!J
/media/term/MISC/Malware/Malimg/malimg_dataset_imgs/Autorun.K
/media/term/MISC/Malware/Malimg/malimg_dataset_imgs/C2LOP.gen!g
/media/term/MISC/Malware/Malimg/malimg_dataset_imgs/C2LOP.P
/media/term/MISC/Malware/Malimg/malimg_dataset_imgs/Dialplatform.B
/media/term/MISC/Malware/Malimg/malimg_dataset_imgs/Dontovo.A
/media/term/MISC/Malware/Malimg/malimg_dataset_imgs/Fakerean
/media/term/MISC/Malware/Malimg/malimg_dataset_imgs/Instantaccess
/media/term/MISC/Malware/Malimg/malimg_dataset_imgs/Lolyda.AA1
/media/term/MISC/Malware/Malimg/malimg_dataset_imgs/Lolyda.AA2
/media/term/MISC/Malware/Malimg/malimg_dataset_imgs/Lolyda.AA3
/media/term/MISC/Malware/Malimg/malimg_dataset_imgs/Lol

In [5]:
import random
from sklearn.cross_validation import StratifiedKFold
from sklearn.utils import shuffle

n_samples, n_features = X.shape 
p = range(n_samples) # an index array, 0:n_samples
random.seed(random.random())
# Shuffling Array Index
random.shuffle(p) 
# Both the arrays are now shuffled
X, y = X[p], y[p] 

# Assigning number of folds
kfold = 10 

skf = StratifiedKFold(y,kfold,indices='true')


skfind = [None]*len(skf)
cnt=0
for train_index in skf:
  skfind[cnt] = train_index
  cnt = cnt + 1

# created skfind list with the following map description
# skfind[i][0] -> train indices, skfind[i][1] -> test indices

In [6]:
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
import time

Initializing the Confusion Matrix
conf_mat = numpy.zeros((len(no_imgs),len(no_imgs))) 

#Assigning Number Of Neighbours
n_neighbors = 1; 

# 10-fold Cross Validation
for i in range(kfold):
 train_indices = skfind[i][0]
 test_indices = skfind[i][1]
 clf = []
 clf = KNeighborsClassifier(n_neighbors,weights='distance') 
 X_train = X[train_indices]
 y_train = y[train_indices]
 X_test = X[test_indices]
 y_test = y[test_indices]
 
 # Training and measuing time   
 tic = time.time()
 clf.fit(X_train,y_train) 
 toc = time.time()
 print "training time= ", toc-tic 
 
 # Testing and measuring time
 y_predict = []
 tic = time.time()
 y_predict = clf.predict(X_test) # Label Outputs
 toc = time.time()
 print "testing time = ", toc-tic

 # Compute confusion matrix
 cm = []
 cm = confusion_matrix(y_test,y_predict)
 conf_mat = conf_mat + cm 
 
#Taking Transpose of Matrix
conf_mat = conf_mat.T
avg_acc = numpy.trace(conf_mat)/sum(no_imgs)
# Normalizing the confusion matrix
conf_mat_norm = conf_mat/no_imgs 

training time=  0.337185144424
testing time =  4.57801008224
training time=  0.285086154938
testing time =  4.5261027813
training time=  0.289582014084
testing time =  4.50061202049
training time=  0.281070947647
testing time =  4.50996184349
training time=  0.279736995697
testing time =  4.48517489433
training time=  0.296093940735
testing time =  4.5204410553
training time=  0.281272888184
testing time =  4.460313797
training time=  0.317667007446
testing time =  4.5412709713
training time=  0.287338972092
testing time =  4.57624793053
training time=  0.28048491478
testing time =  4.48427009583


In [8]:
import pickle
pickle.dump(clf, open("clf_dump", 'wb'))

In [9]:
os.getcwd()

'/media/term/MISC/Malware/Malimg/malimg_dataset_imgs'

In [8]:
print conf_mat_norm

[[1.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [0.00000000e+00 1.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 9.98304510e-01 1.63419233e-02
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.

In [10]:
#using numpy to limit decimal accuracy
conf_mat2 = numpy.around(conf_mat_norm,decimals=2)
print conf_mat2

[[1.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   1.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   1.   0.01 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.01 0.   0.01 0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.99 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.01 0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.99 0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   1.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.87 0.21 0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.01 0.   0.1  0.06 0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.08 0.67 0.   0.   0.   0.   0.   0.
  0.01 0.   0.   0.   0.   0.   0.

In [11]:
print avg_acc

0.9707677481529071


In [13]:
numpy.save("conf_mat2.npy", conf_mat2)

In [14]:
numpy.save("conf_mat_norm.npy", conf_mat_norm)